<a href="https://colab.research.google.com/github/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/TPClassificationLivres.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
---
# Classification de commentaires de livres (en français)

## Collection de données de critiques de films
Le tp utilisera l'ensemble de données issues de
[French Book reviews](https://www.kaggle.com/datasets/abireltaief/books-reviews) que vous devez télécharger.

Ce fichier CSV contient presque 10000 commentaires sur des livres, leurs notes, et l'avis (positif, négatif ou neutre).

---
Vous vous baserez sur la solution à *[Détection de "sentiments"](https://github.com/EmmanuelADAM/IntelligenceArtificiellePython/blob/master/SolutionTPDetectionDeSentiments.ipynb)*.

Vous adapterez le code présent pour traiter le texte en français.
Simpement, les "stopwords" seront chargés ainsi :

In [1]:
from os import listdir
import pandas as pd
from tqdm import tqdm

##import pour la creation de tokens
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize

##import pour la gestion de ponctuation
from string import punctuation

##import pour une collection spécifique : counter
from collections import Counter

##import pour les réseaux de neurones :
import tensorflow.keras as keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

##import classique pour les tableaux efficaces
import numpy as np

In [9]:
nltk.download("stopwords")
stop_words = set(stopwords.words('french'))

Series = pd.core.series.Series
DataFrame = pd.core.frame.DataFrame

df = pd.read_json("hf://datasets/Abirate/french_book_reviews/french_books_reviews.jsonl", lines=True)
reader_reviews:Series = df["reader_review"]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


---
A partir de la colonne 'reader_review' plusieurs classements sont possibles :
- soit le classement de valeurs entre -1 et 1 (ce qui correspond à la colonne 'label')
- soit le classement par une note entre . et 5 (ce qui correspond à la colonne 'rating')

*N.B. la fonction d'activation sigmoid pren ses valeurs dans [0,1], l'activation en utilisant la tangente hyperboliquye 'tanh' prend ses valeus dans [-1, 1]*

In [10]:
def clean_doc(doc:str)->list:
    """retourne la liste de mots clés inclus dans le texte doc
    qui ne font pas parti des stop_words"""
    # split into tokens by white space
    tokens = wordpunct_tokenize(doc)
    # remove remaining tokens that are not alphabetic
    tokens = [word.lower() for word in tokens if word.isalpha()]
    # filter out stop words
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 2]
    return tokens

def add_doc_to_vocab(reader_review:str, vocab:Counter):
    """cumule dans la liste vocab les mots de la review
    (1 seule occurrence par mot dans vocab)"""
    # clean doc
    tokens = clean_doc(reader_review)
    # update counts
    vocab.update(tokens)

def build_voc(serie:Series, vocab:Counter):
    """ajoute au dictionnaire vocab les mots cles de 90% de la serie de données"""
    i=0
    nb = int(serie.size)
    # walk through all files in the folder
    for i in tqdm(range(nb)):
        # add doc to vocab
        add_doc_to_vocab(serie[i], vocab)

# creer un vocabulaire (liste de mots clés associés à leurs occurrences)
vocab:Counter = Counter()
build_voc(reader_reviews, vocab)
# afficher le nb de mots cles trouves
print("nb de mots cles trouves dans les repertoires : ", len(vocab))
print("les 10 premiers mots cles du vocabulaire \
(et leur nb d\'apparition dans les exemples)  : \n", end='')
i:int=0
for (mot,count) in vocab.items():
    print(mot,':',count,end=", ")
    i = i+1
    if i>10:break
# afficher les 10 mots cles les plus utilises
print("\nles 10 mots cles les plus utilises : ", vocab.most_common(10))

100%|██████████| 9658/9658 [00:00<00:00, 11146.56it/s]

nb de mots cles trouves dans les repertoires :  33512
les 10 premiers mots cles du vocabulaire (et leur nb d'apparition dans les exemples)  : 
premier : 666, roman : 2083, aborder : 32, thèmes : 72, lourds : 5, inceste : 6, enfance : 141, martyre : 1, fait : 1339, audace : 10, brio : 22, 
les 10 mots cles les plus utilises :  [('plus', 2435), ('livre', 2380), ('cette', 2223), ('tout', 2097), ('roman', 2083), ('histoire', 1978), ('bien', 1853), ('très', 1814), ('fait', 1339), ('comme', 1273)]
